In [1]:
import imgkit
import logging
import numpy as np
import pandas as pd
import requests
import seaborn as sns

from requests_toolbelt import sessions
logging.basicConfig(
    filename="logs/bot.log",
    format="%(asctime)s - [%(levelname)s] - %(message)s", 
    level=logging.INFO
)

In [2]:
def get_endpoint(endpoint, params=None):
    with sessions.BaseUrlSession(base_url="https://api.covid19api.com/") as session:
        try:
            response = session.get(endpoint)
            response.raise_for_status()
            logging.info(f"Request successful for endpoint={endpoint}.")
        except requests.exceptions.HTTPError as e:
            logging.error(f"{e}. Retrying...")
            response = get_endpoint(endpoint)
    return response

def parse_countries(countries, sort_key, ascending=True):
    table = pd.DataFrame.from_dict(countries)
    table.drop(columns=["CountryCode", "Date", "Slug"], inplace=True)
    cols = [table.columns[idx] for idx in [0, 4, 1, 5, 2, 6, 3]]
    table = table[cols]
    return table.sort_values(sort_key, ascending=ascending)

def split_table(table, size=10):
    return [table[i:i+size] for i in np.arange(0, table.shape[0], size)]

def style_table(table, palette):
    cm = sns.light_palette(palette, as_cmap=True)
    formatter = {col: "{:,d}" for col in table.dtypes[tab.dtypes == np.int64].index}
    table_styles = [
        {
            'selector': '',
            'props': [
                ('width', '100%'),
                ('align', 'center')
            ]
        },
        {
            'selector': 'th, td, .col0', 
            'props': [
                ('background-color', '#ffffff'),
                ('color', '#000000'),
                ('text-align', 'left'),
                ('border', '0px'),
                ('font-family', 'Arial, Helvetica, sans-serif'),
                ('padding', '0.25em')                    
            ]
        },
    ]
    style = table.style \
                 .format(formatter) \
                 .background_gradient(cmap=cm) \
                 .set_table_styles(table_styles)
    
    return style.hide_index().render()

In [3]:
response = get_endpoint("/summary")
date = response.json()["Date"]
table = parse_countries(response.json()["Countries"], "TotalConfirmed", ascending=False)
tables = split_table(table, size=31)
for idx, tab in enumerate(tables, 1):
    html = style_table(tab, "red")
    imgkit.from_string(html, f"output/table_{idx}.png", options={'quiet': ''})